Download dataset

In [1]:
!wget "https://coinmetrics.io/data/eth.csv" -O eth.csv

--2018-09-01 07:37:55--  https://coinmetrics.io/data/eth.csv
Resolving coinmetrics.io (coinmetrics.io)... 139.59.153.246
Connecting to coinmetrics.io (coinmetrics.io)|139.59.153.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190103 (186K) [text/csv]
Saving to: ‘eth.csv’

eth.csv             100%[===================>] 185.65K  --.-KB/s    in 0.09s   

2018-09-01 07:37:55 (2.02 MB/s) - ‘eth.csv’ saved [190103/190103]



Import dependencies

In [2]:
import csv
import dateutil

Load data into an array

In [3]:
eth_data = []

with open('eth.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        eth_data.append(row)

print(",".join(eth_data[0])) # columns
print("\n")

eth_data = eth_data[1:] # remove column names row
print(",".join(eth_data[0])) # print first row

date,txVolume(USD),adjustedTxVolume(USD),txCount,marketcap(USD),price(USD),exchangeVolume(USD),generatedCoins,fees,activeAddresses,medianTxValue(USD),medianFee,averageDifficulty,paymentCount,blockSize,blockCount


2015-07-30,,,,,,,39311.09375,,,,,1.21335493201e+11,,4449897,6911,


Create structured data array

In [4]:
class EthDataPoint:
    def __init__(self, date, price):
        self.date = dateutil.parser.parse(date)
        if price is "":
            self.price = None
        else:
            self.price = float(price)
        
    def __repr__(self):
        return f"EthDataPoint{{date={self.date},price={self.price}}}"

In [5]:
eth_dps_mapped = list(map(lambda row: EthDataPoint(row[0], row[5]), eth_data))
eth_dps = list(filter(lambda dp: dp.price != None, eth_dps_mapped))

Store transactions

In [9]:
class Tx:
    def __init__(self, buyOrSell, amount, price, date, closed):
        self.typ = buyOrSell
        self.amount = amount
        self.price = price
        self.date = date
        self.closed = closed
    
    def __repr__(self):
        return f"Tx{{typ={self.typ},amount={self.amount},price={self.price},date={self.date},closed={self.closed}}}"

Traverse data and generate transactions

In [12]:
txs = []
lookback_days = 7

for i in range(lookback_days, len(eth_dps)):
    start_price = eth_dps[i - lookback_days].price
    end_price = eth_dps[i].price
    diff_pct = round((end_price-start_price)/start_price * 10000) / 100
    print(eth_dps[i].date, diff_pct)
    
    if diff_pct > 20:
        txs.append(Tx("buy", 1, eth_dps[i].price, eth_dps[i].date, False))
    
    for j in range(0, len(txs)):
        if txs[j].typ is "buy" and txs[j].closed is False and (eth_dps[i].date - txs[j].date).days >= 7:
            txs[j].closed = True
            txs.append(Tx("sell", -1, eth_dps[i].price, eth_dps[i].date, True))

2015-08-14 00:00:00 -36.04
2015-08-15 00:00:00 -35.48
2015-08-16 00:00:00 137.91
2015-08-17 00:00:00 121.29
2015-08-18 00:00:00 72.3
2015-08-19 00:00:00 10.38
2015-08-20 00:00:00 2.46
2015-08-21 00:00:00 -18.23
2015-08-22 00:00:00 -22.22
2015-08-23 00:00:00 -17.86
2015-08-24 00:00:00 -14.56
2015-08-25 00:00:00 0.82
2015-08-26 00:00:00 -3.42
2015-08-27 00:00:00 -6.4
2015-08-28 00:00:00 -22.3
2015-08-29 00:00:00 -15.0
2015-08-30 00:00:00 -14.49
2015-08-31 00:00:00 -2.22
2015-09-01 00:00:00 9.76
2015-09-02 00:00:00 19.47
2015-09-03 00:00:00 11.11
2015-09-04 00:00:00 9.57
2015-09-05 00:00:00 7.56
2015-09-06 00:00:00 13.56
2015-09-07 00:00:00 -1.52
2015-09-08 00:00:00 -7.41
2015-09-09 00:00:00 -8.15
2015-09-10 00:00:00 -6.92
2015-09-11 00:00:00 -7.94
2015-09-12 00:00:00 -22.87
2015-09-13 00:00:00 -22.39
2015-09-14 00:00:00 -27.65
2015-09-15 00:00:00 -29.98
2015-09-16 00:00:00 -24.03
2015-09-17 00:00:00 -25.05
2015-09-18 00:00:00 -24.61
2015-09-19 00:00:00 -13.94
2015-09-20 00:00:00 -15.27
2

2018-04-11 00:00:00 -0.35
2018-04-12 00:00:00 13.21
2018-04-13 00:00:00 28.85
2018-04-14 00:00:00 32.99
2018-04-15 00:00:00 30.37
2018-04-16 00:00:00 32.73
2018-04-17 00:00:00 27.97
2018-04-18 00:00:00 21.27
2018-04-19 00:00:00 21.82
2018-04-20 00:00:00 15.17
2018-04-21 00:00:00 25.06
2018-04-22 00:00:00 20.53
2018-04-23 00:00:00 16.75
2018-04-24 00:00:00 25.88
2018-04-25 00:00:00 40.48
2018-04-26 00:00:00 17.95
2018-04-27 00:00:00 16.57
2018-04-28 00:00:00 4.65
2018-04-29 00:00:00 12.83
2018-04-30 00:00:00 11.04
2018-05-01 00:00:00 4.21
2018-05-02 00:00:00 -4.66
2018-05-03 00:00:00 11.08
2018-05-04 00:00:00 17.32
2018-05-05 00:00:00 21.71
2018-05-06 00:00:00 19.33
2018-05-07 00:00:00 15.02
2018-05-08 00:00:00 12.61
2018-05-09 00:00:00 11.69
2018-05-10 00:00:00 9.61
2018-05-11 00:00:00 -6.41
2018-05-12 00:00:00 -13.34
2018-05-13 00:00:00 -15.8
2018-05-14 00:00:00 -7.64
2018-05-15 00:00:00 -3.16
2018-05-16 00:00:00 -5.95
2018-05-17 00:00:00 -5.83
2018-05-18 00:00:00 -7.55
2018-05-19 00:

In [13]:
txs

[Tx{typ=buy,amount=1,price=1.68,date=2015-08-16 00:00:00,closed=True},
 Tx{typ=buy,amount=1,price=1.58,date=2015-08-17 00:00:00,closed=True},
 Tx{typ=buy,amount=1,price=1.22,date=2015-08-18 00:00:00,closed=True},
 Tx{typ=sell,amount=-1,price=1.38,date=2015-08-23 00:00:00,closed=True},
 Tx{typ=sell,amount=-1,price=1.35,date=2015-08-24 00:00:00,closed=True},
 Tx{typ=sell,amount=-1,price=1.23,date=2015-08-25 00:00:00,closed=True},
 Tx{typ=buy,amount=1,price=0.70889,date=2015-10-27 00:00:00,closed=True},
 Tx{typ=buy,amount=1,price=0.870938,date=2015-10-28 00:00:00,closed=True},
 Tx{typ=buy,amount=1,price=1.01,date=2015-10-29 00:00:00,closed=True},
 Tx{typ=buy,amount=1,price=1.21,date=2015-10-30 00:00:00,closed=True},
 Tx{typ=buy,amount=1,price=1.05,date=2015-10-31 00:00:00,closed=True},
 Tx{typ=buy,amount=1,price=0.920847,date=2015-11-01 00:00:00,closed=True},
 Tx{typ=buy,amount=1,price=1.06,date=2015-11-02 00:00:00,closed=True},
 Tx{typ=buy,amount=1,price=0.994568,date=2015-11-03 00:00:00

In [14]:
sum(map(lambda tx: tx.amount * tx.price, txs))

-2856.2013049999996